In [1]:
import sys
sys.path.append('../src')
import Nuesslein1
import Nuesslein2
import CJ1
import CJ2
import utils
from pathlib import Path
import numpy as np

## Generate 3SAT random instances

#### Generate 3 different experiments, i.e. 3 different sets of problems with different number of variables and clauses

In [2]:
secure = input("Do you really want to generate all the problems? (y/n)")
if secure == 'y' or secure == 'Y':
    utils.generate_3_sat_problems()

Do you really want to generate all the problems? (y/n) y


## Number of non-zero couplings in QUBO

#### Generate .txt files with the number of non zero couplings for each gadget and set of instances with the same variables in experiment 1

In [ ]:
gadgets = {
    'Nuesslein1': Nuesslein1,
    'Nuesslein2': Nuesslein2,
    'CJ1': CJ1,
    'CJ2': CJ2,
}
#Information for experiment 1
n_vars = np.arange(20,401,20)
num_instances = 20

for g, g_module in gadgets.items():
    for vars in n_vars:
        dir = f"../exp/e1/Non_0_couplings/{g}/p{vars}"
        p_dir = Path(dir)
        p_dir.mkdir(parents=True, exist_ok=True)
        with open(f"{dir}/non_zero_couplings.txt", 'w') as f:
            non_zero_c=[]
            for i in range(num_instances):
                file_path = f'../exp/e1/problems/p{vars}-{i}.cnf'
                instance = getattr(g_module, g)(file_path)
                instance.fillQ()
                Q = instance.Q
                non_zero_c.append(utils.get_non_zero_coupings(Q))
            f.write(str(non_zero_c))

## Number of physical qubits required

#### Generate .txt files with the embedding for D'Wave architecture for each gadget and each instance in experiment 2

In [2]:
gadgets = {
    'Nuesslein1': Nuesslein1,
    'Nuesslein2': Nuesslein2,
    'CJ1': CJ1,
    'CJ2': CJ2,
}
#Information for experiment 2
n_vars = np.arange(15,28,3)
num_instances = 20

for g, g_module in gadgets.items():
    for vars in n_vars:
        for i in range(num_instances):
            file_path = f'../exp/e2/problems/p{vars}-{i}.cnf'
            instance = getattr(g_module, g)(file_path)
            instance.fillQ()
            embedding = utils.get_embedding(instance.Q)
            if embedding == None:
                print(f"Error while embedding for {g}, vars={vars}, instance={i}")
            dir = f'../exp/e2/Embeddings/{g}/p{vars}'
            p_dir = Path(dir)
            p_dir.mkdir(parents=True, exist_ok=True)
            with open(f'{dir}/p{vars}-{i}.txt', 'w') as archivo:
                archivo.write(str(embedding))

## Solving with Quantum Annealing (D'Wave) and Exact solver (Maxsatz)

#### Generate .txt files with the reponse sample_set obtained and relevant parammeters for each gadget and each instance in experiment 3

In [3]:
gadgets = {
    'Nuesslein1': Nuesslein1,
    'Nuesslein2': Nuesslein2,
    'CJ1': CJ1,
    'CJ2': CJ2,
}
#Information for experiment 3
n_vars=[5,10,12,20,50]
iterations=20

for vars in n_vars:
    for i in range(iterations):
        file_path = f"../exp/e3/problems/p{vars}-{i}.cnf"

        #Exact solver (Maxsatz)
        dir = '../exp/e3/Maxsatz_(exact)'
        p_dir = Path(dir)
        p_dir.mkdir(parents=True, exist_ok=True)
        file_name = f'{dir}/p{vars}-{i}.txt'
        with open(file_name, 'w') as archivo:
            response = !./../src/Maxsatz/maxsatz {file_path}
            for line in response:
                archivo.write(line + '\n')
        
        #Quantum Annealing (D'Wave)
        for g, g_module in gadgets.items():
            dir = f"../exp/e3/DWave/{g}/p{vars}"
            p_dir = Path(dir)
            p_dir.mkdir(parents=True, exist_ok=True)
            file_name = f'{dir}/p{vars}-{i}.txt'
            instance = getattr(g_module, g)(file_path)
            instance.solve(file_name)